# MABe Challenge - XGBoost training Notebook

📝 **Note:** Please note that comments and explanations are in Japanese. However, I've made an effort to write clear, self-explanatory code that should be accessible to non-Japanese speakers as well.

## inference and submission notebook: 
https://www.kaggle.com/code/hutch1221/mabe-starter-inference-ja/notebook

In [1]:
!pip install -q --no-index --find-links=/kaggle/input/mabe-package xgboost==3.1.1

In [2]:
import datetime
import gc
import itertools
import json
import re
import sys
import time
import traceback
from collections import defaultdict
from pathlib import Path

import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedGroupKFold
from tqdm.auto import tqdm

sys.path.append("/kaggle/usr/lib/mabe-f-beta")
from metric import score

In [3]:
# const
INPUT_DIR = Path("/kaggle/input/MABe-mouse-behavior-detection")
TRAIN_TRACKING_DIR = INPUT_DIR / "train_tracking"
TRAIN_ANNOTATION_DIR = INPUT_DIR / "train_annotation"
TEST_TRACKING_DIR = INPUT_DIR / "test_tracking"

WORKING_DIR = Path("/kaggle/working")

INDEX_COLS = [
    "video_id",
    "agent_mouse_id",
    "target_mouse_id",
    "video_frame",
]

BODY_PARTS = [
    "ear_left",
    "ear_right",
    "nose",
    "neck",
    "body_center",
    "lateral_left",
    "lateral_right",
    "hip_left",
    "hip_right",
    "tail_base",
    "tail_tip",
]

SELF_BEHAVIORS = [
    "biteobject",
    "climb",
    "dig",
    "exploreobject",
    "freeze",
    "genitalgroom",
    "huddle",
    "rear",
    "rest",
    "run",
    "selfgroom",
]

PAIR_BEHAVIORS = [
    "allogroom",
    "approach",
    "attack",
    "attemptmount",
    "avoid",
    "chase",
    "chaseattack",
    "defend",
    "disengage",
    "dominance",
    "dominancegroom",
    "dominancemount",
    "ejaculate",
    "escape",
    "flinch",
    "follow",
    "intromit",
    "mount",
    "reciprocalsniff",
    "shepherd",
    "sniff",
    "sniffbody",
    "sniffface",
    "sniffgenital",
    "submit",
    "tussle",
]

In [4]:
# read data
train_dataframe = pl.read_csv(INPUT_DIR / "train.csv")

In [5]:
# preprocess behavior labels
train_behavior_dataframe = (
    train_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
    )
    .explode("behaviors_labeled_list")
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled_element").str.split(",").list[0].str.replace_all("'", "").alias("agent"),
        pl.col("behaviors_labeled_element").str.split(",").list[1].str.replace_all("'", "").alias("target"),
        pl.col("behaviors_labeled_element").str.split(",").list[2].str.replace_all("'", "").alias("behavior"),
    )
)

train_self_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(SELF_BEHAVIORS))
train_pair_behavior_dataframe = train_behavior_dataframe.filter(pl.col("behavior").is_in(PAIR_BEHAVIORS))

## 特徴量加工(self)
- agentの体の各部位間距離(cm)

  主要な11部位(定数: BODY_PARTS)の組み合わせの距離
  
- 部位の推定速度(cm/s)

  ear_left, ear_right, tail_base の 500, 1000, 2000, 3000ms間の推定速度
  
- 伸長度

  (nose-tail_baseの距離) / (ear_left-ear_rightの距離)

- 体角度(deg)
  
  nose-body_center, body_center-tail_base　がなすベクトルの角度

In [6]:
%%writefile self_features.py

def make_self_features(
    metadata: dict,
    tracking: pl.DataFrame,
) -> pl.DataFrame:
    def body_parts_distance(body_part_1, body_part_2):
        # agentの体の各部位間距離(cm)
        assert body_part_1 in BODY_PARTS
        assert body_part_2 in BODY_PARTS
        return (
            (pl.col(f"agent_x_{body_part_1}") - pl.col(f"agent_x_{body_part_2}")).pow(2)
            + (pl.col(f"agent_y_{body_part_1}") - pl.col(f"agent_y_{body_part_2}")).pow(2)
        ).sqrt() / metadata["pix_per_cm_approx"]

    def body_part_speed(body_part, period_ms):
        # 部位の推定速度(cm/s)
        assert body_part in BODY_PARTS
        window_frames = max(1, int(round(period_ms * metadata["frames_per_second"] / 1000.0)))
        return (
            ((pl.col(f"agent_x_{body_part}").diff()).pow(2) + (pl.col(f"agent_y_{body_part}").diff()).pow(2)).sqrt()
            / metadata["pix_per_cm_approx"]
            * metadata["frames_per_second"]
        ).rolling_mean(window_size=window_frames, center=True, min_samples=1)

    def elongation():
        # 伸長度
        d1 = body_parts_distance("nose", "tail_base")
        d2 = body_parts_distance("ear_left", "ear_right")
        return d1 / (d2 + 1e-06)

    def body_angle():
        # 体角度(deg)
        v1x = pl.col("agent_x_nose") - pl.col("agent_x_body_center")
        v1y = pl.col("agent_y_nose") - pl.col("agent_y_body_center")
        v2x = pl.col("agent_x_tail_base") - pl.col("agent_x_body_center")
        v2y = pl.col("agent_y_tail_base") - pl.col("agent_y_body_center")
        return (v1x * v2x + v1y * v2y) / ((v1x.pow(2) + v1y.pow(2)).sqrt() * (v2x.pow(2) + v2y.pow(2)).sqrt() + 1e-06)

    n_mice = (
        (metadata["mouse1_strain"] is not None)
        + (metadata["mouse2_strain"] is not None)
        + (metadata["mouse3_strain"] is not None)
        + (metadata["mouse4_strain"] is not None)
    )
    start_frame = tracking.select(pl.col("video_frame").min()).item()
    end_frame = tracking.select(pl.col("video_frame").max()).item()

    result = []

    pivot = tracking.pivot(
        on=["bodypart"],
        index=["video_frame", "mouse_id"],
        values=["x", "y"],
    ).sort(["mouse_id", "video_frame"])
    pivot_trackings = {mouse_id: pivot.filter(pl.col("mouse_id") == mouse_id) for mouse_id in range(1, n_mice + 1)}

    for agent_mouse_id in range(1, n_mice + 1):
        result_element = pl.DataFrame(
            {
                "video_id": metadata["video_id"],
                "agent_mouse_id": agent_mouse_id,
                "target_mouse_id": -1,
                "video_frame": pl.arange(start_frame, end_frame + 1, eager=True),
            },
            schema={
                "video_id": pl.Int32,
                "agent_mouse_id": pl.Int8,
                "target_mouse_id": pl.Int8,
                "video_frame": pl.Int32,
            },
        )

        pivot = pivot_trackings[agent_mouse_id].select(
            pl.col("video_frame"),
            pl.exclude("video_frame").name.prefix("agent_"),
        )
        columns = pivot.columns
        pivot = pivot.with_columns(
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_x_{bp}") for bp in BODY_PARTS if f"agent_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_y_{bp}") for bp in BODY_PARTS if f"agent_y_{bp}" not in columns],
        )

        features = pivot.with_columns(
            pl.lit(agent_mouse_id).alias("agent_mouse_id"),
            pl.lit(-1).alias("target_mouse_id"),
        ).select(
            pl.col("video_frame"),
            pl.col("agent_mouse_id"),
            pl.col("target_mouse_id"),
            *[
                body_parts_distance(body_part_1, body_part_2).alias(f"aa__{body_part_1}__{body_part_2}__distance")
                for body_part_1, body_part_2 in itertools.combinations(BODY_PARTS, 2)
            ],
            *[
                body_part_speed(body_part, period_ms).alias(f"agent__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            elongation().alias("agent__elongation"),
            body_angle().alias("agent__body_angle"),
        )

        result_element = result_element.join(
            features,
            on=["video_frame", "agent_mouse_id", "target_mouse_id"],
            how="left",
        )
        result.append(result_element)

    return pl.concat(result, how="vertical")

Writing self_features.py


## 特徴量加工(pair)
- agent-target の体の各部位間距離(cm)

  agent-target の主要な11部位(定数: BODY_PARTS)の組み合わせの距離
  
- agent, target の部位の推定速度(cm/s)

  ear_left, ear_right, tail_base の 500, 1000, 2000, 3000ms間の推定速度
  
- agent, target の伸長度

  (nose-tail_baseの距離) / (ear_left-ear_rightの距離)

- agent, target の体角度(deg)
  
  nose-body_center, body_center-tail_base　がなすベクトルの角度

In [7]:
%%writefile pair_features.py

def make_pair_features(
    metadata: dict,
    tracking: pl.DataFrame,
) -> pl.DataFrame:
    def body_parts_distance(agent_or_target_1, body_part_1, agent_or_target_2, body_part_2):
        # agent-targetの体の各部位間距離(cm)
        assert agent_or_target_1 == "agent" or agent_or_target_1 == "target"
        assert agent_or_target_2 == "agent" or agent_or_target_2 == "target"
        assert body_part_1 in BODY_PARTS
        assert body_part_2 in BODY_PARTS
        return (
            (pl.col(f"{agent_or_target_1}_x_{body_part_1}") - pl.col(f"{agent_or_target_2}_x_{body_part_2}")).pow(2)
            + (pl.col(f"{agent_or_target_1}_y_{body_part_1}") - pl.col(f"{agent_or_target_2}_y_{body_part_2}")).pow(2)
        ).sqrt() / metadata["pix_per_cm_approx"]

    def body_part_speed(agent_or_target, body_part, period_ms):
        # 部位の推定速度(cm/s)
        assert agent_or_target == "agent" or agent_or_target == "target"
        assert body_part in BODY_PARTS
        window_frames = max(1, int(round(period_ms * metadata["frames_per_second"] / 1000.0)))
        return (
            (
                (pl.col(f"{agent_or_target}_x_{body_part}").diff()).pow(2)
                + (pl.col(f"{agent_or_target}_y_{body_part}").diff()).pow(2)
            ).sqrt()
            / metadata["pix_per_cm_approx"]
            * metadata["frames_per_second"]
        ).rolling_mean(window_size=window_frames, center=True)

    def elongation(agent_or_target):
        # 伸長度(cm)
        assert agent_or_target == "agent" or agent_or_target == "target"
        d1 = body_parts_distance(agent_or_target, "nose", agent_or_target, "tail_base")
        d2 = body_parts_distance(agent_or_target, "ear_left", agent_or_target, "ear_right")
        return d1 / (d2 + 1e-06)

    def body_angle(agent_or_target):
        # 体角度(deg)
        assert agent_or_target == "agent" or agent_or_target == "target"
        v1x = pl.col(f"{agent_or_target}_x_nose") - pl.col(f"{agent_or_target}_x_body_center")
        v1y = pl.col(f"{agent_or_target}_y_nose") - pl.col(f"{agent_or_target}_y_body_center")
        v2x = pl.col(f"{agent_or_target}_x_tail_base") - pl.col(f"{agent_or_target}_x_body_center")
        v2y = pl.col(f"{agent_or_target}_y_tail_base") - pl.col(f"{agent_or_target}_y_body_center")
        return (v1x * v2x + v1y * v2y) / ((v1x.pow(2) + v1y.pow(2)).sqrt() * (v2x.pow(2) + v2y.pow(2)).sqrt() + 1e-06)

    def body_center_distance_rolling_agg(agg, period_ms):
        # 距離の移動集計特徴量
        assert agg in ["mean", "std", "var", "min", "max"] # 集計関数
        expr = body_parts_distance("agent", "body_center", "target", "body_center")
        window_frames = max(1, int(round(period_ms * metadata["frames_per_second"] / 1000.0)))

        if agg == "mean":
            return expr.rolling_mean(window_size=window_frames, center=True, min_samples=1)
        elif agg == "std":
            return expr.rolling_std(window_size=window_frames, center=True, min_samples=1)
        elif agg == "var":
            return expr.rolling_var(window_size=window_frames, center=True, min_samples=1)
        elif agg == "min":
            return expr.rolling_min(window_size=window_frames, center=True, min_samples=1)
        elif agg == "max":
            return expr.rolling_max(window_size=window_frames, center=True, min_samples=1)
        else:
            raise ValueError()

    n_mice = (
        (metadata["mouse1_strain"] is not None)
        + (metadata["mouse2_strain"] is not None)
        + (metadata["mouse3_strain"] is not None)
        + (metadata["mouse4_strain"] is not None)
    )
    start_frame = tracking.select(pl.col("video_frame").min()).item()
    end_frame = tracking.select(pl.col("video_frame").max()).item()

    result = []

    pivot = tracking.pivot(
        on=["bodypart"],
        index=["video_frame", "mouse_id"],
        values=["x", "y"],
    ).sort(["mouse_id", "video_frame"])
    pivot_trackings = {mouse_id: pivot.filter(pl.col("mouse_id") == mouse_id) for mouse_id in range(1, n_mice + 1)}

    for agent_mouse_id, target_mouse_id in itertools.permutations(range(1, n_mice + 1), 2):
        result_element = pl.DataFrame(
            {
                "video_id": metadata["video_id"],
                "agent_mouse_id": agent_mouse_id,
                "target_mouse_id": target_mouse_id,
                "video_frame": pl.arange(start_frame, end_frame + 1, eager=True),
            },
            schema={
                "video_id": pl.Int32,
                "agent_mouse_id": pl.Int8,
                "target_mouse_id": pl.Int8,
                "video_frame": pl.Int32,
            },
        )

        merged_pivot = (
            pivot_trackings[agent_mouse_id]
            .select(
                pl.col("video_frame"),
                pl.exclude("video_frame").name.prefix("agent_"),
            )
            .join(
                pivot_trackings[target_mouse_id].select(
                    pl.col("video_frame"),
                    pl.exclude("video_frame").name.prefix("target_"),
                ),
                on="video_frame",
                how="inner",
            )
        )
        columns = merged_pivot.columns
        merged_pivot = merged_pivot.with_columns(
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_x_{bp}") for bp in BODY_PARTS if f"agent_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"agent_y_{bp}") for bp in BODY_PARTS if f"agent_y_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"target_x_{bp}") for bp in BODY_PARTS if f"target_x_{bp}" not in columns],
            *[pl.lit(None).cast(pl.Float32).alias(f"target_y_{bp}") for bp in BODY_PARTS if f"target_y_{bp}" not in columns],
        )

        features = merged_pivot.with_columns(
            pl.lit(agent_mouse_id).alias("agent_mouse_id"),
            pl.lit(target_mouse_id).alias("target_mouse_id"),
        ).select(
            pl.col("video_frame"),
            pl.col("agent_mouse_id"),
            pl.col("target_mouse_id"),
            *[
                body_parts_distance("agent", agent_body_part, "target", target_body_part).alias(
                    f"at__{agent_body_part}__{target_body_part}__distance"
                )
                for agent_body_part, target_body_part in itertools.product(BODY_PARTS, repeat=2)
            ],
            *[
                body_part_speed("agent", body_part, period_ms).alias(f"agent__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            *[
                body_part_speed("target", body_part, period_ms).alias(f"target__{body_part}__speed_{period_ms}ms")
                for body_part, period_ms in itertools.product(["ear_left", "ear_right", "tail_base"], [500, 1000, 2000, 3000])
            ],
            elongation("agent").alias("agent__elongation"),
            elongation("target").alias("target__elongation"),
            body_angle("agent").alias("agent__body_angle"),
            body_angle("target").alias("target__body_angle"),
        )

        result_element = result_element.join(
            features,
            on=["video_frame", "agent_mouse_id", "target_mouse_id"],
            how="left",
        )
        result.append(result_element)

    return pl.concat(result, how="vertical")

Writing pair_features.py


In [8]:
%run -i self_features.py
%run -i pair_features.py

def process_video(row):
    """Process a single video to extract self and pair features."""
    lab_id = row["lab_id"]
    video_id = row["video_id"]

    tracking_path = TRAIN_TRACKING_DIR / f"{lab_id}/{video_id}.parquet"
    tracking = pl.read_parquet(tracking_path)

    self_features = make_self_features(metadata=row, tracking=tracking)
    pair_features = make_pair_features(metadata=row, tracking=tracking)

    self_features.write_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
    pair_features.write_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")

    return video_id


# make data
(WORKING_DIR / "self_features").mkdir(exist_ok=True, parents=True)
(WORKING_DIR / "pair_features").mkdir(exist_ok=True, parents=True)

rows = list(train_dataframe.filter(pl.col("behaviors_labeled").is_not_null()).rows(named=True))
results = joblib.Parallel(n_jobs=-1, verbose=5)(joblib.delayed(process_video)(row) for row in rows)

print(f"Processed {len(results)} videos successfully")

del rows, results
gc.collect()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 848 out of 848 | elapsed:  5.7min finished


Processed 848 videos successfully


27

In [9]:
def tune_threshold(oof_action, y_action):
    thresholds = np.arange(0, 1.005, 0.005)
    scores = [f1_score(y_action, (oof_action >= th), zero_division=0) for th in thresholds]
    best_idx = np.argmax(scores)
    return thresholds[best_idx]

## 学習・検証

lab, behavior 毎にモデル(XGBoost)を作る

クロスバリデーション(3fold)でf1スコアを計算しながら学習する

In [10]:
def train_validate(lab_id: str, behavior: str, indices: pl.DataFrame, features: pl.DataFrame, labels: pl.Series):
    # 結果を保存するディレクトリのパスを作成
    result_dir = WORKING_DIR / "results" / lab_id / behavior
    # ディレクトリが存在しない場合は作成（親ディレクトリも含めて）
    result_dir.mkdir(exist_ok=True, parents=True)

    # ラベルの合計が0の場合（正例が1つもない場合）の処理
    if labels.sum() == 0:
        # F1スコアを0として保存
        with open(result_dir / "f1.txt", "w") as f:
            f.write("0.0\n")
        # すべての予測値を0とした結果データフレームを作成
        oof_prediction_dataframe = indices.with_columns(
            pl.Series("fold", [-1] * len(labels), dtype=pl.Int8),  # フォールド番号（-1は未使用を意味）
            pl.Series("prediction", [0.0] * len(labels), dtype=pl.Float32),  # 予測確率
            pl.Series("predicted_label", [0] * len(labels), dtype=pl.Int8),  # 予測ラベル
        )
        # 結果をparquet形式で保存
        oof_prediction_dataframe.write_parquet(result_dir / "oof_predictions.parquet")
        return 0.0

    # Out-of-Fold予測結果を保存するための配列を初期化
    folds = np.ones(len(labels), dtype=np.int8) * -1  # 各サンプルが属するフォールド番号
    oof_predictions = np.zeros(len(labels), dtype=np.float32)  # 予測確率
    oof_prediction_labels = np.zeros(len(labels), dtype=np.int8)  # 予測ラベル（0または1）

    # 3分割の層化グループ交差検証を実行
    # StratifiedGroupKFoldは、ラベルの分布を保ちつつ、同じグループ（video_id）が複数のフォールドに分かれないようにする
    for fold, (train_idx, valid_idx) in enumerate(
        StratifiedGroupKFold(n_splits=3, shuffle=True, random_state=42).split(
            X=features,  # 特徴量
            y=labels,  # ラベル
            groups=indices.get_column("video_id"),  # グループ化の基準（同じ動画IDは同じフォールドに）
        )
    ):
        # 各フォールドの結果を保存するディレクトリを作成
        result_dir_fold = result_dir / f"fold_{fold}"
        result_dir_fold.mkdir(exist_ok=True, parents=True)

        # 訓練データと検証データに分割
        X_train = features[train_idx]  # 訓練用特徴量
        y_train = labels[train_idx]  # 訓練用ラベル
        X_valid = features[valid_idx]  # 検証用特徴量
        y_valid = labels[valid_idx]  # 検証用ラベル

        # クラス不均衡に対処するための重みを計算
        # 負例の数 / 正例の数 = 正例にかける重み
        scale_pos_weight = (len(y_train) - y_train.sum()) / y_train.sum()

        # XGBoostのハイパーパラメータを設定
        params = {
            "objective": "binary:logistic",  # 二値分類問題
            "eval_metric": "logloss",  # 評価指標：対数損失
            "device": "cpu",  # 使用デバイス
            "tree_method": "hist",  # ヒストグラムベースの高速なアルゴリズム
            "learning_rate": 0.05,  # 学習率
            "max_depth": 6,  # 木の最大深さ
            "min_child_weight": 5,  # 子ノードの最小重み
            "subsample": 0.8,  # 各木で使用するサンプルの割合
            "colsample_bytree": 0.8,  # 各木で使用する特徴量の割合
            "scale_pos_weight": scale_pos_weight,  # 正例の重み
            "max_bin": 64,  # ヒストグラムのビン数
            "seed": 42,  # 乱数シード
        }
        
        # XGBoost用のデータ行列を作成（訓練データは量子化行列、検証データは通常の行列）
        dtrain = xgb.QuantileDMatrix(X_train, label=y_train, feature_names=features.columns, max_bin=64)
        dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_names=features.columns)

        # 評価結果を保存する辞書
        evals_result = {}
        
        # 早期終了のコールバックを設定
        # 検証データの対数損失が10ラウンド改善しない場合、学習を停止
        early_stopping_callback = xgb.callback.EarlyStopping(
            rounds=10,  # 改善が見られない連続ラウンド数
            metric_name="logloss",  # 監視する指標
            data_name="valid",  # 監視するデータセット
            maximize=False,  # 小さい方が良い指標
            save_best=True,  # 最良のモデルを保存
        )
        
        # モデルの学習を実行
        model = xgb.train(
            params,  # ハイパーパラメータ
            dtrain=dtrain,  # 訓練データ
            num_boost_round=250,  # 最大ブースティングラウンド数
            evals=[(dtrain, "train"), (dvalid, "valid")],  # 評価するデータセット
            callbacks=[early_stopping_callback],  # コールバック
            evals_result=evals_result,  # 評価結果の保存先
            verbose_eval=0,  # ログ出力の頻度（0は出力なし）
        )

        # 検証データに対して予測を実行（確率値を取得）
        fold_predictions = model.predict(dvalid)

        # F1スコアを最大化する最適な閾値を調整
        threshold = tune_threshold(fold_predictions, y_valid)
        
        # Out-of-Fold予測結果を保存
        folds[valid_idx] = fold  # フォールド番号
        oof_predictions[valid_idx] = fold_predictions  # 予測確率
        oof_prediction_labels[valid_idx] = (fold_predictions >= threshold).astype(np.int8)  # 閾値で二値化

        # このフォールドの結果を保存
        # 学習済みモデルを保存
        model.save_model(result_dir_fold / "model.json")
        # 最適な閾値を保存
        with open(result_dir_fold / "threshold.txt", "w") as f:
            f.write(f"{threshold}\n")

        # 特徴量の重要度をプロット（上位20個、ゲイン基準）
        xgb.plot_importance(model, max_num_features=20, importance_type="gain", values_format="{v:.2f}")
        plt.tight_layout()
        plt.savefig(result_dir_fold / "feature_importance.png")
        plt.close()

        # 学習曲線（対数損失の推移）をプロット
        lgb.plot_metric(evals_result, metric="logloss")
        plt.tight_layout()
        plt.savefig(result_dir_fold / "metric.png")
        plt.close()

        # メモリを解放
        gc.collect()

    # すべてのフォールドの予測結果をデータフレームにまとめる
    oof_prediction_dataframe = indices.with_columns(
        pl.Series("fold", folds, dtype=pl.Int8),  # フォールド番号
        pl.Series("prediction", oof_predictions, dtype=pl.Float32),  # 予測確率
        pl.Series("predicted_label", oof_prediction_labels, dtype=pl.Int8),  # 予測ラベル
    )
    
    # 全体のF1スコアを計算
    f1 = f1_score(labels, oof_prediction_labels, zero_division=0)
    # F1スコアをファイルに保存
    with open(result_dir / "f1.txt", "w") as f:
        f.write(f"{f1}\n")

    # 予測結果データフレームを保存
    oof_prediction_dataframe.write_parquet(result_dir / "oof_predictions.parquet")

    # F1スコアを返す
    return f1

## (self) lab-behavior毎に学習-検証を行う

In [11]:
groups = train_self_behavior_dataframe.group_by("lab_id", "behavior", maintain_order=True)
total_groups = len(list(groups))
start_time = time.perf_counter()

for idx, ((lab_id, behavior), group) in tqdm(enumerate(groups), total=total_groups):
    if idx == 0:
        tqdm.write(
            f"|{'LAB':^25}|{'BEHAVIOR':^15}|{'SAMPLES':^10}|{'POSITIVE':^10}|{'FEATURES':^10}|{'F1':^10}|{'ELAPSED TIME':^15}|",
            end="\n",
        )

    tqdm.write(f"|{lab_id:^25}|{behavior:^15}|", end="")
    index_list = []
    feature_list = []
    label_list = []

    for row in group.rows(named=True):
        video_id = row["video_id"]
        agent = row["agent"]

        agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))

        data = pl.scan_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet").filter(
            (pl.col("agent_mouse_id") == agent_mouse_id)
        )
        index = data.select(INDEX_COLS).collect(engine="streaming")
        feature = data.select(pl.exclude(INDEX_COLS)).collect(engine="streaming")

        # read annotation
        annotation_path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        if annotation_path.exists():
            annotation = (
                pl.scan_parquet(annotation_path)
                .filter((pl.col("action") == behavior) & (pl.col("agent_id") == agent_mouse_id))
                .collect()
            )
        else:
            annotation = pl.DataFrame(
                schema={
                    "agent_id": pl.Int8,
                    "target_id": pl.Int8,
                    "action": str,
                    "start_frame": pl.Int16,
                    "stop_frame": pl.Int16,
                }
            )

        label_frames = set()
        for annotation_row in annotation.rows(named=True):
            label_frames.update(range(annotation_row["start_frame"], annotation_row["stop_frame"]))
        label = index.select(pl.col("video_frame").is_in(label_frames).cast(pl.Int8).alias("label"))

        if label.get_column("label").sum() == 0:
            continue

        index_list.append(index)
        feature_list.append(feature)
        label_list.append(label.get_column("label"))

    if not index_list:
        elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
        tqdm.write(f"{0:>10,}|{0:>10,}|{0:>10,}|{'-':>10}|{str(elapsed_time):>15}|", end="\n")
        continue

    indices = pl.concat(index_list, how="vertical")
    features = pl.concat(feature_list, how="vertical")
    labels = pl.concat(label_list, how="vertical")

    del index_list, feature_list, label_list
    gc.collect()

    tqdm.write(f"{len(indices):>10,}|{labels.sum():>10,}|{len(features.columns):>10,}|", end="")

    f1 = train_validate(lab_id, behavior, indices, features, labels)
    tqdm.write(f"{f1:>10.2f}|", end="")

    elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
    tqdm.write(f"{str(elapsed_time):>15}|", end="\n")

    gc.collect()

  0%|          | 0/27 [00:00<?, ?it/s]

|           LAB           |   BEHAVIOR    | SAMPLES  | POSITIVE | FEATURES |    F1    | ELAPSED TIME  |
|     AdaptableSnail      |     rear      |   660,348|    85,313|        69|      0.62|        0:01:40|
|         CRIM13          |     rear      |   179,132|    12,042|        69|      0.36|        0:02:09|
|         CRIM13          |   selfgroom   |   205,533|    14,472|        69|      0.36|        0:02:42|
|      CalMS21_task1      | genitalgroom  |   102,445|     6,270|        69|      0.66|        0:02:59|
|       ElegantMink       |     rear      |         0|         0|         0|         -|        0:03:00|
|       ElegantMink       |   selfgroom   |         0|         0|         0|         -|        0:03:00|
|       GroovyShrew       |     rear      |   899,280|    50,768|        69|      0.53|        0:04:50|
|       GroovyShrew       |     rest      |   530,886|    87,573|        69|      0.68|        0:05:40|
|       GroovyShrew       |   selfgroom   |   877,773|    22,893

## (pair) lab-behavior毎に学習-検証を行う

In [12]:
groups = train_pair_behavior_dataframe.group_by("lab_id", "behavior", maintain_order=True)
total_groups = len(list(groups))
start_time = time.perf_counter()

for idx, ((lab_id, behavior), group) in tqdm(enumerate(groups), total=total_groups):
    if idx == 0:
        tqdm.write(
            f"|{'LAB':^25}|{'BEHAVIOR':^15}|{'SAMPLES':^10}|{'POSITIVE':^10}|{'FEATURES':^10}|{'F1':^10}|{'ELAPSED TIME':^15}|",
            end="\n",
        )

    tqdm.write(f"|{lab_id:^25}|{behavior:^15}|", end="")
    index_list = []
    feature_list = []
    label_list = []

    for row in group.rows(named=True):
        video_id = row["video_id"]
        agent = row["agent"]
        target = row["target"]

        agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))
        target_mouse_id = int(re.search(r"mouse(\d+)", target).group(1))

        data = pl.scan_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet").filter(
            (pl.col("agent_mouse_id") == agent_mouse_id) & (pl.col("target_mouse_id") == target_mouse_id)
        )
        index = data.select(INDEX_COLS).collect(engine="streaming")
        feature = data.select(pl.exclude(INDEX_COLS)).collect(engine="streaming")

        # read annotation
        annotation_path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        if annotation_path.exists():
            annotation = (
                pl.scan_parquet(annotation_path)
                .filter(
                    (pl.col("action") == behavior)
                    & (pl.col("agent_id") == agent_mouse_id)
                    & (pl.col("target_id") == target_mouse_id)
                )
                .collect()
            )
        else:
            annotation = pl.DataFrame(
                schema={
                    "agent_id": pl.Int8,
                    "target_id": pl.Int8,
                    "action": str,
                    "start_frame": pl.Int16,
                    "stop_frame": pl.Int16,
                }
            )

        label_frames = set()
        for annotation_row in annotation.rows(named=True):
            label_frames.update(range(annotation_row["start_frame"], annotation_row["stop_frame"]))
        label = index.select(pl.col("video_frame").is_in(label_frames).cast(pl.Int8).alias("label"))

        if label.get_column("label").sum() == 0:
            continue

        index_list.append(index)
        feature_list.append(feature)
        label_list.append(label.get_column("label"))

    if not index_list:
        elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
        tqdm.write(f"{0:>10,}|{0:>10,}|{0:>10,}|{'-':>10}|{str(elapsed_time):>15}|", end="\n")
        continue

    indices = pl.concat(index_list, how="vertical")
    features = pl.concat(feature_list, how="vertical")
    labels = pl.concat(label_list, how="vertical")

    del index_list, feature_list, label_list
    gc.collect()

    tqdm.write(f"{len(indices):>10,}|{labels.sum():>10,}|{len(features.columns):>10,}|", end="")

    f1 = train_validate(lab_id, behavior, indices, features, labels)
    tqdm.write(f"{f1:>10.2f}|", end="")

    elapsed_time = datetime.timedelta(seconds=int(time.perf_counter() - start_time))
    tqdm.write(f"{str(elapsed_time):>15}|", end="\n")

    gc.collect()

  0%|          | 0/104 [00:00<?, ?it/s]

|           LAB           |   BEHAVIOR    | SAMPLES  | POSITIVE | FEATURES |    F1    | ELAPSED TIME  |
|     AdaptableSnail      |   approach    | 1,524,536|     8,083|       149|      0.35|        0:04:58|
|     AdaptableSnail      |    attack     | 2,436,568|    28,759|       149|      0.18|        0:11:47|
|     AdaptableSnail      |     avoid     | 5,538,087|    22,923|       149|      0.17|        0:28:58|
|     AdaptableSnail      |     chase     | 3,707,542|    14,739|       149|      0.14|        0:39:40|
|     AdaptableSnail      |  chaseattack  | 1,217,344|     4,157|       149|      0.26|        0:43:20|
|     AdaptableSnail      |    submit     |   424,181|     8,478|       149|      0.39|        0:45:03|
|    BoisterousParrot     |   shepherd    | 9,504,414|    29,451|       149|      0.49|        1:12:37|
|         CRIM13          |   approach    |   205,533|    10,178|       149|      0.49|        1:13:19|
|         CRIM13          |    attack     |    71,906|     7,594

In [13]:
%%writefile robustify.py

def robustify(submission: pl.DataFrame, dataset: pl.DataFrame, train_test: str = "train"):
    traintest_directory = INPUT_DIR / f"{train_test}_tracking"

    old_submission = submission.clone()
    submission = submission.filter(pl.col("start_frame") < pl.col("stop_frame"))
    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")

    old_submission = submission.clone()
    group_list = []
    for _, group in submission.group_by("video_id", "agent_id", "target_id"):
        group = group.sort("start_frame")
        mask = np.ones(len(group), dtype=bool)
        last_stop_frame = 0
        for i, row in enumerate(group.rows(named=True)):
            if row["start_frame"] < last_stop_frame:
                mask[i] = False
            else:
                last_stop_frame = row["stop_frame"]
        group_list.append(group.filter(pl.Series("mask", mask)))

    submission = pl.concat(group_list)

    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")

    s_list = []
    for row in dataset.rows(named=True):
        lab_id = row["lab_id"]
        video_id = row["video_id"]
        if row["behaviors_labeled"] is None:
            continue

        if video_id in submission.get_column("video_id").to_list():
            continue

        if isinstance(row["behaviors_labeled"], str):
            continue

        print(f"Video {video_id} has no predictions.")

        path = traintest_directory / f"/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)

        vid_behaviors = json.loads(row["behaviors_labeled"])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(",") for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=["agent", "target", "action"])

        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1

        for (agent, target), actions in vid_behaviors.groupby(["agent", "target"]):
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, action_row in enumerate(actions.itertuples(index=False)):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)
                s_list.append((video_id, agent, target, action_row["action"], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat(
            [
                submission,
                pd.DataFrame(s_list, columns=["video_id", "agent_id", "target_id", "action", "start_frame", "stop_frame"]),
            ]
        )
        print("ERROR: Filled empty videos")

    return submission

Writing robustify.py


## 検証データに対する予測値を集計

In [14]:
# グループごとのOut-of-Fold予測結果を保存するリスト
group_oof_predictions = []

# データを lab_id, video_id, agent, target でグループ化
# maintain_order=True で元の順序を保持
groups = train_behavior_dataframe.group_by("lab_id", "video_id", "agent", "target", maintain_order=True)

# 各グループに対して処理を実行（進捗バーを表示）
for (lab_id, video_id, agent, target), group in tqdm(groups, total=len(list(groups))):
    # agent（行動主体）からマウスIDを抽出
    # 例: "mouse1" → 1
    agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))
    
    # target（行動対象）からマウスIDを抽出
    # "self"（自分自身）の場合は -1、それ以外はマウスIDを取得
    target_mouse_id = -1 if target == "self" else int(re.search(r"mouse(\d+)", target).group(1))

    # このグループの各行動の予測結果を保存するリスト
    prediction_dataframe_list = []

    # グループ内の各行（各行動）を処理
    for row in group.rows(named=True):
        behavior = row["behavior"]  # 行動の種類（例: "grooming", "sniffing"など）

        # この行動のOOF予測結果ファイルのパスを構築
        oof_path = WORKING_DIR / "results" / lab_id / behavior / "oof_predictions.parquet"
        
        # ファイルが存在しない場合はスキップ
        if not oof_path.exists():
            continue

        # 予測結果を読み込み、該当するvideo_id、agent、targetでフィルタリング
        prediction = (
            pl.scan_parquet(oof_path)  # 遅延読み込み（メモリ効率的）
            .filter(
                (pl.col("video_id") == video_id)  # 動画IDが一致
                & (pl.col("agent_mouse_id") == agent_mouse_id)  # 行動主体が一致
                & (pl.col("target_mouse_id") == target_mouse_id)  # 行動対象が一致
            )
            .select(
                *INDEX_COLS,  # インデックス列を選択
                # 予測確率と予測ラベルを掛け合わせて、この行動のスコアを計算
                # 予測ラベルが0の場合はスコアも0になる
                (pl.col("prediction") * pl.col("predicted_label")).alias(behavior)
            )
            .collect()  # 実際にデータを読み込んで実行
        )

        # フィルタ後に行がない場合（該当データがない場合）はスキップ
        if len(prediction) == 0:
            continue

        # この行動の予測結果をリストに追加
        prediction_dataframe_list.append(prediction)

    # このグループで予測結果が1つもない場合はスキップ
    if not prediction_dataframe_list:
        continue

    # 複数の行動の予測結果を横方向に結合
    # how="align"で、インデックス列を基準に整列して結合
    prediction_dataframe = pl.concat(prediction_dataframe_list, how="align")

    # インデックス列以外の列名（各行動名）を取得
    cols = prediction_dataframe.select(pl.exclude(INDEX_COLS)).columns
    
    # 各フレームで最も確信度の高い行動を選択
    prediction_labels_dataframe = prediction_dataframe.with_columns(
        pl.struct(pl.exclude(INDEX_COLS))  # 全行動のスコアを構造体にまとめる
        .map_elements(
            # 各行に対して実行する関数
            lambda row: "none" if sum(row.values()) == 0  # 全スコアが0なら"none"
                       else (cols[np.argmax(list(row.values()))]),  # 最大スコアの行動を選択
            return_dtype=pl.String,
        )
        .alias("prediction")  # 新しい列名を"prediction"とする
    ).select(INDEX_COLS + ["prediction"])  # インデックス列と予測列のみを選択

    # 連続する同じ行動をまとめて、行動の開始と終了フレームを特定
    group_oof_prediction = (
        prediction_labels_dataframe
        .filter((pl.col("prediction") != pl.col("prediction").shift(1)))  # 前の行と異なる行動のみを抽出（境界点）
        .with_columns(pl.col("video_frame").shift(-1).alias("stop_frame"))  # 次の境界点を終了フレームとする
        .filter(pl.col("prediction") != "none")  # "none"（行動なし）を除外
        .select(
            pl.col("video_id"),  # 動画ID
            ("mouse" + pl.col("agent_mouse_id").cast(str)).alias("agent_id"),  # "mouse1"形式に変換
            # target_mouse_idが-1なら"self"、それ以外は"mouse2"形式に変換
            pl.when(pl.col("target_mouse_id") == -1)
            .then(pl.lit("self"))
            .otherwise("mouse" + pl.col("target_mouse_id").cast(str))
            .alias("target_id"),
            pl.col("prediction").alias("action"),  # 行動名
            pl.col("video_frame").alias("start_frame"),  # 開始フレーム
            pl.col("stop_frame"),  # 終了フレーム
        )
    )

    # このグループの予測結果をリストに追加
    group_oof_predictions.append(group_oof_prediction)

%run -i robustify.py

oof_predictions = pl.concat(group_oof_predictions, how="vertical")
oof_predictions = robustify(oof_predictions, train_dataframe, train_test="train")
oof_predictions.with_row_index("row_id").write_csv(WORKING_DIR / "oof_predictions.csv")

  0%|          | 0/1442 [00:00<?, ?it/s]

ERROR: Dropped frames with start >= stop


<Figure size 640x480 with 0 Axes>

## 検証データによるスコアを計算する

In [15]:

def compute_validation_metrics(submission, verbose=True):
    """Compute and display validation metrics for single vs pair behaviors."""
    # solution_df
    dataset = pl.read_csv(INPUT_DIR / "train.csv").to_pandas()

    solution = []
    for _, row in dataset.iterrows():
        lab_id = row["lab_id"]
        if lab_id.startswith("MABe22"):
            continue

        video_id = row["video_id"]
        path = TRAIN_ANNOTATION_DIR / lab_id / f"{video_id}.parquet"
        try:
            annot = pd.read_parquet(path)
        except FileNotFoundError:
            continue

        annot["lab_id"] = lab_id
        annot["video_id"] = video_id
        annot["behaviors_labeled"] = row["behaviors_labeled"]
        annot["target_id"] = np.where(
            annot.target_id != annot.agent_id, annot["target_id"].apply(lambda s: f"mouse{s}"), "self"
        )
        annot["agent_id"] = annot["agent_id"].apply(lambda s: f"mouse{s}")
        solution.append(annot)

    solution = pd.concat(solution)

    try:
        # Separate single and pair behaviors
        submission_single = submission[submission["target_id"] == "self"].copy()
        submission_pair = submission[submission["target_id"] != "self"].copy()

        # Filter solution to match submission videos
        solution_videos = set(submission["video_id"].unique())
        solution = solution[solution["video_id"].isin(solution_videos)]

        if len(solution) == 0:
            return

        # Compute overall F1 score
        overall_f1 = score(solution, submission, "row_id", beta=1.0)
        print(f"\n{'=' * 60}")
        print("PERFORMANCE METRICS")
        print(f"{'=' * 60}")
        print(f"Overall F1 Score: {overall_f1:.4f}")
        print(f"Total predictions: {len(submission)}")
        print(f"  - Single behaviors: {len(submission_single)}")
        print(f"  - Pair behaviors: {len(submission_pair)}")

        # Compute per-action F1 scores using existing scoring function
        solution_pl = pl.DataFrame(solution)
        submission_pl = pl.DataFrame(submission)

        # Add label_key and prediction_key
        solution_pl = solution_pl.with_columns(
            pl.concat_str(
                [
                    pl.col("video_id").cast(pl.Utf8),
                    pl.col("agent_id").cast(pl.Utf8),
                    pl.col("target_id").cast(pl.Utf8),
                    pl.col("action"),
                ],
                separator="_",
            ).alias("label_key"),
        )
        submission_pl = submission_pl.with_columns(
            pl.concat_str(
                [
                    pl.col("video_id").cast(pl.Utf8),
                    pl.col("agent_id").cast(pl.Utf8),
                    pl.col("target_id").cast(pl.Utf8),
                    pl.col("action"),
                ],
                separator="_",
            ).alias("prediction_key"),
        )

        # Group by action and compute metrics
        action_stats = defaultdict(lambda: {"single": {"count": 0, "f1": 0.0}, "pair": {"count": 0, "f1": 0.0}})

        for lab in solution_pl["lab_id"].unique():
            lab_solution = solution_pl.filter(pl.col("lab_id") == lab).clone()
            lab_videos = set(lab_solution["video_id"].unique())
            lab_submission = submission_pl.filter(pl.col("video_id").is_in(lab_videos)).clone()

            # Compute per-action F1 using same logic as single_lab_f1
            label_frames = defaultdict(set)
            prediction_frames = defaultdict(set)

            for row in lab_solution.to_dicts():
                label_frames[row["label_key"]].update(range(row["start_frame"], row["stop_frame"]))

            for row in lab_submission.to_dicts():
                key = row["prediction_key"]
                prediction_frames[key].update(range(row["start_frame"], row["stop_frame"]))

            for key in set(list(label_frames.keys()) + list(prediction_frames.keys())):
                action = key.split("_")[-1]
                mode = "single" if "self" in key else "pair"

                pred_frames = prediction_frames.get(key, set())
                label_frames_set = label_frames.get(key, set())

                tp = len(pred_frames & label_frames_set)
                fn = len(label_frames_set - pred_frames)
                fp = len(pred_frames - label_frames_set)

                if tp + fn + fp > 0:
                    f1 = (1 + 1**2) * tp / ((1 + 1**2) * tp + 1**2 * fn + fp)
                    action_stats[action][mode]["count"] += 1
                    action_stats[action][mode]["f1"] += f1

        # Print per-action summary
        print("\nPer-Action Performance Summary:")
        print(f"{'-' * 60}")
        print(f"{'Action':<20} {'Mode':<10} {'Count':<10} {'Avg F1':<10}")
        print(f"{'-' * 60}")

        for action in sorted(action_stats.keys()):
            for mode in ["single", "pair"]:
                stats = action_stats[action][mode]
                if stats["count"] > 0:
                    avg_f1 = stats["f1"] / stats["count"]
                    print(f"{action:<20} {mode:<10} {stats['count']:<10} {avg_f1:<10.4f}")

        # Summary by mode
        single_actions = [a for a in action_stats.keys() if action_stats[a]["single"]["count"] > 0]
        pair_actions = [a for a in action_stats.keys() if action_stats[a]["pair"]["count"] > 0]

        if single_actions:
            single_avg_f1 = np.mean(
                [
                    action_stats[a]["single"]["f1"] / action_stats[a]["single"]["count"]
                    for a in single_actions
                    if action_stats[a]["single"]["count"] > 0
                ]
            )
            print(f"\nSingle behaviors: {len(single_actions)} actions, Avg F1: {single_avg_f1:.4f}")

        if pair_actions:
            pair_avg_f1 = np.mean(
                [
                    action_stats[a]["pair"]["f1"] / action_stats[a]["pair"]["count"]
                    for a in pair_actions
                    if action_stats[a]["pair"]["count"] > 0
                ]
            )
            print(f"Pair behaviors: {len(pair_actions)} actions, Avg F1: {pair_avg_f1:.4f}")

        print(f"{'=' * 60}\n")

    except Exception as e:
        if verbose:
            error_msg = str(e)
            if len(error_msg) > 200:
                error_msg = error_msg[:200] + "..."
            print(f"\nWarning: Could not compute validation metrics: {error_msg}")
            if verbose:
                print(f"Traceback: {traceback.format_exc()[:300]}")

compute_validation_metrics(submission=pd.read_csv(WORKING_DIR / "oof_predictions.csv"))


PERFORMANCE METRICS
Overall F1 Score: 0.5041
Total predictions: 455285
  - Single behaviors: 113613
  - Pair behaviors: 341672

Per-Action Performance Summary:
------------------------------------------------------------
Action               Mode       Count      Avg F1    
------------------------------------------------------------
allogroom            pair       17         0.1645    
approach             pair       258        0.3714    
attack               pair       389        0.5583    
attemptmount         pair       42         0.1183    
avoid                pair       136        0.1507    
biteobject           single     16         0.0229    
chase                pair       117        0.1379    
chaseattack          pair       22         0.1737    
climb                single     30         0.3894    
defend               pair       64         0.3910    
dig                  single     60         0.3364    
disengage            pair       20         0.4296    
dominance      